# Building and Training Neural Networks in TensorFlow and Keras

In this project, we will build and train neural networks using the TensorFlow library and the Keras front-end to the library. We will figure out how to transform our sentiment and handwriting data into `numpy` arrays that can serve as inputs to a TensorFlow neural network. We will then employ the tools provided by Keras to train and assess the networks.

In [1]:
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from typing import *
import random

In [2]:
def assess(value: Any, expected: Any) -> bool:
    print(value)
    return value == expected

def assess_np(np_value: np.ndarray, np_expected: np.ndarray) -> bool:
    print(np_value)
    return (np_value == np_expected).all()

def assess_np_list(np_values: List[np.ndarray], np_expecteds: List[np.ndarray]) -> bool:
    assert len(np_values) == len(np_expecteds)
    for i in range(len(np_values)):
        if not assess_np(np_values[i], np_expecteds[i]):
            return False
    return True

# Level 1: Processing Training Inputs
* Processing Sentiment Examples
* Processing Drawing Examples

## Processing Sentiment Examples

Processing sentiment data requires the following steps:
* Remove all punctuation and make all letters lower case.
  * `lower_alpha_only()`
* Transform each text example into a bag of words.
  * `bag_of_words()`
* The network needs to have a unique input node for each unique term. To do this, we:
  * Find all unique terms across the entire data set.
    * `all_words_from()`
  * Create a table giving the input node index for each word.
    * `word_index_table()`
* Read examples from a file into bags of words.
  * `sentiment_data_from()`
* Transform bags of words into neural network inputs.
  * `net_ready_sentiment_data()`
* Transform files into neural network inputs
  * `net_sentiment_data_from()` (provided)

In [3]:
def lower_alpha_only(s: str) -> str:
    s = s.lower()
    output = ""
    for c in s:
        if c.isalpha() or c == ' ':
            output += c
    return output

In [4]:
assess(lower_alpha_only('This is a Test.'), "this is a test")

this is a test


True

In [5]:
def bag_of_words(string: str) -> Dict[str,int]:
    parsed = lower_alpha_only(string)
    ws = parsed.split()
    d = dict()
    for w in ws:
        x = 1
        if w in d:
            x += d[w]
        d[w] = x
    return d

In [6]:
assess(bag_of_words('This is a test. This is only a test.'), {'this': 2, 'is': 2, 'a': 2, 'test': 2, 'only': 1})

{'this': 2, 'is': 2, 'a': 2, 'test': 2, 'only': 1}


True

In [7]:
import os

def sentiment_data_from(filename: str) -> Tuple[List[Dict[str,int]], List[int]]:
    data_path = os.path.join(filename)
    with open(data_path, 'r', encoding='utf-8') as data_file:
        data = data_file.readlines()
    listdictoutput = []
    listintoutput  = []
    for d in data:
        bag = bag_of_words(lower_alpha_only(d))
        if '1' in d:            
            listintoutput.append(1)  
            listdictoutput.append(bag) 
        elif '0' in d:
            listintoutput.append(0)  
            listdictoutput.append(bag)     
    return (listdictoutput, listintoutput)

In [8]:
assess(sentiment_data_from('../input/csci-335-example/sentiment_demo_1.txt'),
       ([{'this': 1, 'is': 1, 'a': 1, 'happy': 1,  'day': 1},
         {'this': 1, 'is': 1, 'a': 1, 'cloudy': 1, 'day': 1},
         {'it': 1, 'is': 1, 'interesting': 1, 'to': 1, 'program': 1, 'a': 1, 'circuit': 1},
         {'the': 1, 'weather': 1, 'is': 1, 'very': 1, 'chilly': 1}
        ],
        [1, 0, 1, 0]))

([{'this': 1, 'is': 1, 'a': 1, 'happy': 1, 'day': 1}, {'this': 1, 'is': 1, 'a': 1, 'cloudy': 1, 'day': 1}, {'it': 1, 'is': 1, 'interesting': 1, 'to': 1, 'program': 1, 'a': 1, 'circuit': 1}, {'the': 1, 'weather': 1, 'is': 1, 'very': 1, 'chilly': 1}], [1, 0, 1, 0])


True

In [9]:
def all_words_from(sentiment_examples: List[Dict[str,int]]) -> List[str]:
    output = set()
    for d in sentiment_examples:
        for s in d.keys():
            output.add(s)
    return sorted(list(output))

In [10]:
assess(all_words_from(sentiment_data_from('../input/csci-335-example/sentiment_demo_1.txt')[0]),
      ['a', 'chilly', 'circuit', 'cloudy', 'day', 'happy', 'interesting', 'is', 'it', 'program', 'the', 'this', 'to', 'very', 'weather'])

['a', 'chilly', 'circuit', 'cloudy', 'day', 'happy', 'interesting', 'is', 'it', 'program', 'the', 'this', 'to', 'very', 'weather']


True

In [11]:
def word_index_table(sorted_words: List[str]) -> Dict[str,int]:
    d = {}
    for i in range(len(sorted_words)):
        d[sorted_words[i]] = i
    return d

In [12]:
assess(word_index_table(['a','chilly','circuit','cloudy','day','happy','interesting','is','it','program','the','this','to','very','weather']),
       {'a': 0,'chilly': 1,'circuit': 2,'cloudy': 3,'day': 4,'happy': 5,'interesting': 6,'is': 7,'it': 8,'program': 9,'the': 10,'this': 11,'to': 12,'very': 13,'weather': 14})

{'a': 0, 'chilly': 1, 'circuit': 2, 'cloudy': 3, 'day': 4, 'happy': 5, 'interesting': 6, 'is': 7, 'it': 8, 'program': 9, 'the': 10, 'this': 11, 'to': 12, 'very': 13, 'weather': 14}


True

In [13]:
def net_ready_sentiment_data(sentiment_data: Tuple[List[Dict[str,int]], List[int]]) -> Tuple[np.ndarray, np.ndarray]:
    listdict,binaryclassifictions = sentiment_data[0], sentiment_data[1]
    awf = all_words_from(sentiment_data[0])
    wit = word_index_table(awf)
    a = np.zeros((len(sentiment_data[1]), len(awf)))
    b = np.zeros(len(sentiment_data[1]))
    for idx,dsi in enumerate(sentiment_data[0]):
        for s in dsi.keys():
            a[idx][wit[s]] = float(dsi[s])
    for i in range(len(sentiment_data[1])):
        b[i] = float(sentiment_data[1][i])
    assert len(sentiment_data[0]) == len(sentiment_data[1])
    return (a, b)

In [14]:
test_numpy_sentiment = np.array([[1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1.]])

assess_np_list(net_ready_sentiment_data(sentiment_data_from('../input/csci-335-example/sentiment_demo_1.txt')),
       (test_numpy_sentiment, np.array([1., 0., 1., 0.])))

[[1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1.]]
[1. 0. 1. 0.]


True

In [15]:
test_numpy_sentiment_2 = np.array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 1., 0., 0., 0., 0., 2., 0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 2.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2.],
        [0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1.,
         1., 0., 0., 1., 1., 1., 0., 2., 0., 0., 0., 0.],
        [1., 0., 0., 0., 2., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 2.,
         0., 0., 0., 1., 1., 0., 0., 0., 2., 2., 0., 0.]])
assess_np_list(net_ready_sentiment_data(sentiment_data_from('../input/csci-335-example/sentiment_demo_2.txt')),
               (test_numpy_sentiment_2, np.array([1., 0., 0., 1.])))

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 2. 0.
  1. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 2. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 2.]
 [0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 2.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 2. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 2. 0. 0. 0. 1. 1. 0. 0. 0.
  2. 2. 0. 0.]]
[1. 0. 0. 1.]


True

In [16]:
def net_sentiment_data_from(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    return net_ready_sentiment_data(sentiment_data_from(filename))

## Processing Drawing Examples

Processing handwriting data requires the following steps:
* Read examples from a file into lists of strings.
  * `basic_drawing_data_from()`
* Transform lists of strings into neural network inputs.
  * `net_ready_drawing_data()`
* Transform files into neural network inputs
  * `drawing_data_from()` (provided)

In [17]:
# Return a list of strings, where each string is the source of a drawing.
# Also return a list of labels
#
# These will then be easier to turn into a numpy array.
def basic_drawing_data_from(filename: str) -> Tuple[List[str], List[str]]:
    data_path = os.path.join(filename)
    with open(data_path, 'r', encoding='utf-8') as data_file:
        data = data_file.readlines()
    listdrawingoutput = []
    listlabeloutput  = []
    for d in data:
        label, *drawings = d.split(':')
        for drawing in drawings:
            listdrawingoutput.append(drawing)
            listlabeloutput.append(label)
    return listdrawingoutput, listlabeloutput

In [18]:
test_drawings, test_drawing_labels = basic_drawing_data_from('../input/csci-335-example/drawing_demo_2.txt')
assess(test_drawings, 
       ['3|3|OXX|OOO|XXX',
        '3|3|OOX|XOX|XOO',
        '3|3|OXO|XXX|XOO',
        '3|3|XXO|OXO|OXO',
        '3|3|XOO|OXX|OXX'])
assess(test_drawing_labels, ['Y', 'Y', 'Y', 'Z', 'Z'])

['3|3|OXX|OOO|XXX', '3|3|OOX|XOX|XOO', '3|3|OXO|XXX|XOO\n', '3|3|XXO|OXO|OXO', '3|3|XOO|OXX|OXX\n']
['Y', 'Y', 'Y', 'Z', 'Z']


True

In [19]:
test_drawings, test_drawing_labels = basic_drawing_data_from('../input/csci-335-example/drawing_demo_1.txt')
assess(test_drawings, 
       ['3|3|XXX|OOO|XXO',
        '3|3|XOX|OXO|OXX',
        '3|3|OXO|OXO|XOX',
        '3|3|XOX|XOX|XOX',
        '3|3|OOO|XXX|OOO'])
assess(test_drawing_labels, ['A', 'A', 'B', 'B', 'B'])

['3|3|XXX|OOO|XXO', '3|3|XOX|OXO|OXX\n', '3|3|OXO|OXO|XOX', '3|3|XOX|XOX|XOX', '3|3|OOO|XXX|OOO\n']
['A', 'A', 'B', 'B', 'B']


True

In [20]:
def net_ready_drawing_data(drawings: List[str], labels: List[str]) -> Tuple[np.ndarray, np.ndarray]:
    assert len(drawings) == len(labels)
    x = drawings[0].split('|')
    w, h = int(x[0]), int(x[1])
    drs = np.zeros((len(labels), h, w), dtype='float64')
    lbs = np.zeros(len(labels), dtype='float64')
    dsi = {}
    for a, (drawing, label) in enumerate(zip(drawings, labels)):
        lst = drawing.split('|')[2:]
        if label not in dsi:
            dsi[label] = float(len(dsi.keys()))
        lbs[a] = dsi[label]            
        for b,string in enumerate(lst):
            for c, char in enumerate(string):
                if 'X' == char:
                    drs[a][b][c] = 1.0
    return drs,lbs

In [21]:
net_test_drawings, net_test_drawing_labels = net_ready_drawing_data(test_drawings, test_drawing_labels)
expected_test_drawings = np.array([[[1., 1., 1.],
                                    [0., 0., 0.],
                                    [1., 1., 0.]],
 
                                   [[1., 0., 1.],
                                    [0., 1., 0.],
                                    [0., 1., 1.]],
 
                                   [[0., 1., 0.],
                                    [0., 1., 0.],
                                    [1., 0., 1.]],
 
                                   [[1., 0., 1.],
                                    [1., 0., 1.],
                                    [1., 0., 1.]],
 
                                   [[0., 0., 0.],
                                    [1., 1., 1.],
                                    [0., 0., 0.]]])
assess_np(net_test_drawings, expected_test_drawings)

[[[1. 1. 1.]
  [0. 0. 0.]
  [1. 1. 0.]]

 [[1. 0. 1.]
  [0. 1. 0.]
  [0. 1. 1.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 1.]]

 [[1. 0. 1.]
  [1. 0. 1.]
  [1. 0. 1.]]

 [[0. 0. 0.]
  [1. 1. 1.]
  [0. 0. 0.]]]


True

In [22]:
assess_np(net_test_drawing_labels, np.array([0., 0., 1., 1., 1.]))

[0. 0. 1. 1. 1.]


True

In [23]:
def drawing_data_from(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    drawing_strs, label_strs = basic_drawing_data_from(filename)
    return net_ready_drawing_data(drawing_strs, label_strs)

# Level 2: Setting up and running experiments
* Creating Cross-Validation Sets
* Experiments 
  * Five network architectures for each of:
    * Sentiment Analysis
    * Handwriting Recognition

## Creating Cross-Validation Sets

Creating cross-validation sets requires the following steps:
* Generate a random ordering of examples and labels.
  * `shuffle_indices()` (provided)
* Given an ordering, rearrange a `numpy` array
  * `permuted_numpy()` 
* Given numpy arrays of examples and labels, create cross-validation partitions
  * `make_cross_validation_partitions()`
* Merge a list of `numpy` arrays into a single giant `numpy` array
  * `np_consolidate()` (provided)
* Given cross-validation partitions, make the cross-validation sets
  * `make_cross_validation_sets()`
  
The following functions are all provided for you, but you should carefully examine them. Some of them
show how the functions above will be used, while `evaluate()` in particular is an important function for you to use.
* Make cross-validation sets, given a means of generating data from a file
  * `cross_from()` 
* Make cross-validation sets from a drawing file
  * `drawing_cross_from_file()`
* Make cross-validation sets from a sentiment file
  * `sentiment_cross_from_file()`
* Create and test a neural network using the cross-validation sets
  * `evaluate()`

In [24]:
def shuffle_indices(size: int) -> List[int]:
    indices = [i for i in range(size)]
    random.shuffle(indices)
    return indices

In [25]:
def permuted_numpy(indices: List[int], array: np.ndarray):
    p = list(array)
    for r, i in enumerate(indices):
        p[i] = (array[r])
    return p

def assess_np_list(np_values: List[np.ndarray], np_expecteds: List[np.ndarray]) -> bool:
    assert len(np_values) == len(np_expecteds)
    for i in range(len(np_values)):
        if not assess_np(np_values[i], np_expecteds[i]):
            return False
    return True



In [26]:
test_permutation = [4, 2, 1, 3, 0]
expected_permutation = [np.array([[[0., 0., 0.],
                                   [1., 1., 1.],
                                   [0., 0., 0.]],

                                 [[0., 1., 0.],
                                  [0., 1., 0.],
                                  [1., 0., 1.]],

                                 [[1., 0., 1.],
                                  [0., 1., 0.],
                                  [0., 1., 1.]],

                                 [[1., 0., 1.],
                                  [1., 0., 1.],
                                  [1., 0., 1.]],

                                 [[1., 1., 1.],
                                  [0., 0., 0.],
                                  [1., 1., 0.]]]),
                        np.array([1., 1., 0., 1., 0.])]
permuted_test_drawings = permuted_numpy(test_permutation, net_test_drawings)
permuted_labels = permuted_numpy(test_permutation, net_test_drawing_labels)
assess_np_list([permuted_test_drawings, permuted_labels], expected_permutation)

[array([[0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.]]), array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 1.]]), array([[1., 0., 1.],
       [0., 1., 0.],
       [0., 1., 1.]]), array([[1., 0., 1.],
       [1., 0., 1.],
       [1., 0., 1.]]), array([[1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 0.]])]
[1.0, 1.0, 0.0, 1.0, 0.0]


True

In [27]:
assess_np_list([permuted_test_drawings, permuted_labels], expected_permutation)

[array([[0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.]]), array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 1.]]), array([[1., 0., 1.],
       [0., 1., 0.],
       [0., 1., 1.]]), array([[1., 0., 1.],
       [1., 0., 1.],
       [1., 0., 1.]]), array([[1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 0.]])]
[1.0, 1.0, 0.0, 1.0, 0.0]


True

In [28]:
def make_cross_validation_partitions(n: int, examples: np.ndarray, labels: np.ndarray) -> List[Tuple[np.ndarray, np.ndarray]]:
    assert len(examples) == len(labels)
    size = len(examples) // n
    lens = [size] * n
    output = []
    for s in range(len(examples) % n):
        lens[s] += 1
    a = 0
    for i in lens:
            output.append((examples[a:a+i],labels[a:a+i]))
            a += i
    return output

In [29]:
make_cross_validation_partitions(3, np.array([1,2,3,4,5,6,7]), np.array([1,2,3,4,5,6,7]))

[(array([1, 2, 3]), array([1, 2, 3])),
 (array([4, 5]), array([4, 5])),
 (array([6, 7]), array([6, 7]))]

In [30]:
print(make_cross_validation_partitions(4, permuted_test_drawings, permuted_labels))

[([array([[0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.]]), array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 1.]])], [1.0, 1.0]), ([array([[1., 0., 1.],
       [0., 1., 0.],
       [0., 1., 1.]])], [0.0]), ([array([[1., 0., 1.],
       [1., 0., 1.],
       [1., 0., 1.]])], [1.0]), ([array([[1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 0.]])], [0.0])]


In [31]:
test_partitions = make_cross_validation_partitions(4, permuted_test_drawings, permuted_labels)
print(permuted_test_drawings)
print(permuted_labels)
print(assess_np(test_partitions[0][0], np.array([[[0., 0., 0.],
        [1., 1., 1.],
        [0., 0., 0.]],
       [[0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 1.]]])))
print(assess_np(test_partitions[0][1], np.array([1., 1.])))
print(assess_np(test_partitions[1][0], np.array([[[1., 0., 1.],
        [0., 1., 0.],
        [0., 1., 1.]]])))
print(assess_np(test_partitions[1][1], np.array([0.])))
print(assess_np(test_partitions[2][0], np.array([[[1., 0., 1.],
        [1., 0., 1.],
        [1., 0., 1.]]])))
print(assess_np(test_partitions[2][1], np.array([1.])))
print(assess_np(test_partitions[3][0], np.array([[[1., 1., 1.],
        [0., 0., 0.],
        [1., 1., 0.]]])))
print(assess_np(test_partitions[3][1], np.array([0.])))

[array([[0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.]]), array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 1.]]), array([[1., 0., 1.],
       [0., 1., 0.],
       [0., 1., 1.]]), array([[1., 0., 1.],
       [1., 0., 1.],
       [1., 0., 1.]]), array([[1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 0.]])]
[1.0, 1.0, 0.0, 1.0, 0.0]
[array([[0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.]]), array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 1.]])]
True
[1.0, 1.0]
True
[array([[1., 0., 1.],
       [0., 1., 0.],
       [0., 1., 1.]])]
True
[0.0]
True
[array([[1., 0., 1.],
       [1., 0., 1.],
       [1., 0., 1.]])]
True
[1.0]
True
[array([[1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 0.]])]
True
[0.0]
True


In [32]:
def np_consolidate(arrays: List[np.ndarray]) -> np.ndarray:
    result = np.concatenate((arrays[0], arrays[1]))
    for i in range(2, len(arrays)):
        result = np.concatenate((result, arrays[i]))
    return result    

In [33]:
assess_np(np_consolidate([p[0] for p in test_partitions[:1] + test_partitions[2:]]),np.array([[[0., 0., 0.],
        [1., 1., 1.],
        [0., 0., 0.]],

       [[0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 1.]],

       [[1., 0., 1.],
        [1., 0., 1.],
        [1., 0., 1.]],

       [[1., 1., 1.],
        [0., 0., 0.],
        [1., 1., 0.]]]))

[[[0. 0. 0.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 1.]]

 [[1. 0. 1.]
  [1. 0. 1.]
  [1. 0. 1.]]

 [[1. 1. 1.]
  [0. 0. 0.]
  [1. 1. 0.]]]


True

In [34]:
def make_cross_validation_sets(partitions: List[Tuple[np.ndarray, np.ndarray]]) -> List[Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]]:
    cross_validation_sets = []
    for i in range(len(partitions)):
        validation_partition = np.array(partitions[i][0]), np.array(partitions[i][1])
        training_partitions = partitions[:i] + partitions[i+1:]
        examples = []
        labels   = []
        for x in range(len(training_partitions)): 
            examples.append(training_partitions[x][0])
            labels += [training_partitions[x][1]]
        examples = np.concatenate(examples, axis=0)
        labels = np.concatenate(labels, axis=0)
        cross_validation_sets.append(((np.array(examples), np.array(labels)), validation_partition))
    return cross_validation_sets

In [35]:
test_cross = make_cross_validation_sets(test_partitions)

print(assess_np_list(test_cross[0][0], (np.array([[[1., 0., 1.],
           [0., 1., 0.],
           [0., 1., 1.]],
   
          [[1., 0., 1.],
           [1., 0., 1.],
           [1., 0., 1.]],
   
          [[1., 1., 1.],
           [0., 0., 0.],
           [1., 1., 0.]]]),
   np.array([0., 1., 0.]))))



print(assess_np_list(test_cross[0][1], (np.array([[[0., 0., 0.],
           [1., 1., 1.],
           [0., 0., 0.]],
   
          [[0., 1., 0.],
           [0., 1., 0.],
           [1., 0., 1.]]]),
   np.array([1., 1.]))))

print(assess_np_list(test_cross[1][0], (np.array([[[0., 0., 0.],
           [1., 1., 1.],
           [0., 0., 0.]],
   
          [[0., 1., 0.],
           [0., 1., 0.],
           [1., 0., 1.]],
   
          [[1., 0., 1.],
           [1., 0., 1.],
           [1., 0., 1.]],
   
          [[1., 1., 1.],
           [0., 0., 0.],
           [1., 1., 0.]]]),
   np.array([1., 1., 1., 0.]))))

print(assess_np_list(test_cross[1][1], (np.array([[[1., 0., 1.],
           [0., 1., 0.],
           [0., 1., 1.]]]),
   np.array([0.]))))

print(assess_np_list(test_cross[2][0], (np.array([[[0., 0., 0.],
           [1., 1., 1.],
           [0., 0., 0.]],
   
          [[0., 1., 0.],
           [0., 1., 0.],
           [1., 0., 1.]],
   
          [[1., 0., 1.],
           [0., 1., 0.],
           [0., 1., 1.]],
   
          [[1., 1., 1.],
           [0., 0., 0.],
           [1., 1., 0.]]]), np.array([1., 1., 0., 0.]))))

print(assess_np_list(test_cross[2][1], (np.array([[[1., 0., 1.],
           [1., 0., 1.],
           [1., 0., 1.]]]),
   np.array([1.]))))

print(assess_np_list(test_cross[3][0], (np.array([[[0., 0., 0.],
           [1., 1., 1.],
           [0., 0., 0.]],
   
          [[0., 1., 0.],
           [0., 1., 0.],
           [1., 0., 1.]],
   
          [[1., 0., 1.],
           [0., 1., 0.],
           [0., 1., 1.]],
   
          [[1., 0., 1.],
           [1., 0., 1.],
           [1., 0., 1.]]]),
   np.array([1., 1., 0., 1.]))))

print(assess_np_list(test_cross[3][1], (np.array([[[1., 1., 1.],
           [0., 0., 0.],
           [1., 1., 0.]]]),
   np.array([0.]))))


[[[1. 0. 1.]
  [0. 1. 0.]
  [0. 1. 1.]]

 [[1. 0. 1.]
  [1. 0. 1.]
  [1. 0. 1.]]

 [[1. 1. 1.]
  [0. 0. 0.]
  [1. 1. 0.]]]
[0. 1. 0.]
True
[[[0. 0. 0.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 1.]]]
[1. 1.]
True
[[[0. 0. 0.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 1.]]

 [[1. 0. 1.]
  [1. 0. 1.]
  [1. 0. 1.]]

 [[1. 1. 1.]
  [0. 0. 0.]
  [1. 1. 0.]]]
[1. 1. 1. 0.]
True
[[[1. 0. 1.]
  [0. 1. 0.]
  [0. 1. 1.]]]
[0.]
True
[[[0. 0. 0.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 1.]]

 [[1. 0. 1.]
  [0. 1. 0.]
  [0. 1. 1.]]

 [[1. 1. 1.]
  [0. 0. 0.]
  [1. 1. 0.]]]
[1. 1. 0. 0.]
True
[[[1. 0. 1.]
  [1. 0. 1.]
  [1. 0. 1.]]]
[1.]
True
[[[0. 0. 0.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 1.]]

 [[1. 0. 1.]
  [0. 1. 0.]
  [0. 1. 1.]]

 [[1. 0. 1.]
  [1. 0. 1.]
  [1. 0. 1.]]]
[1. 1. 0. 1.]
True
[[[1. 1. 1.]
  [0. 0. 0.]
  [1. 1. 0.]]]
[0.]
True


In [36]:
def cross_from(n: int, filename: str, data_func) -> List[Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]]:
    examples, labels = data_func(filename)
    indices = shuffle_indices(len(examples))
    permuted_examples = permuted_numpy(indices, examples)
    permuted_labels = permuted_numpy(indices, labels)
    partitions = make_cross_validation_partitions(n, permuted_examples, permuted_labels)
    return make_cross_validation_sets(partitions)

In [37]:
def drawing_cross_from_file(n: int, filename: str) -> List[Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]]:
    return cross_from(n, filename, drawing_data_from)

In [38]:
def evaluate(cross_data: List[Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]], 
             epochs: int, 
             model_maker):
    num_labels = len(set(cross_data[0][0][1]))
    for ((train_examples, train_labels), (test_examples, test_labels)) in cross_data:
        print(f"shape:{train_examples.shape}")
        model = model_maker(train_examples.shape[1:], num_labels)
        model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
        history = model.fit(train_examples, train_labels, epochs=epochs, validation_data=(test_examples, test_labels))  

In [39]:
def sentiment_cross_from_file(n: int, filename: str) -> List[Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]]:
    return cross_from(n, filename, net_sentiment_data_from)

## Experiments

Experiment with at least five different network architectures for each of the sentiment and handwriting tasks. The code block below shows how to use `evaluate()` to assess a network architecture. That example involves two hiden layers, one with 40 nodes and the other with 10 nodes. Select a number of epochs for each run such that the network seems to converge on a particular accuracy level.

Be very careful in recording and analyzing your results. The `accuracy` values are for the **training set**; the `val_accuracy` values are for the **testing set**. In your analysis, be sure to comment on `accuracy`, but since that is for the training set, the main focus of your analysis should be on the `val_accuracy` values that inform you about the testing set outcomes.

In [40]:
# amazon_cross = sentiment_cross_from_file(4, '../input/csci-335-example/amazon_cells_labelled.txt')

In [41]:
# evaluate(amazon_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [42]:
# evaluate(amazon_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [43]:
# evaluate(amazon_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [44]:
# evaluate(amazon_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(80, activation="relu"),
#     keras.layers.Dense(70, activation="relu"),
#     keras.layers.Dense(60, activation="relu"),
#     keras.layers.Dense(50, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(30, activation="relu"),
#     keras.layers.Dense(20, activation="relu"),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [45]:
sarcasm_cross = sentiment_cross_from_file(4, '../input/big-data-internet-aint-safe/sarcasm_labelled.txt')

In [46]:
# evaluate(sarcasm_cross, 60, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [47]:
# evaluate(sarcasm_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [48]:
# evaluate(sarcasm_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [49]:
# evaluate(sarcasm_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(80, activation="relu"),
#     keras.layers.Dense(70, activation="relu"),
#     keras.layers.Dense(60, activation="relu"),
#     keras.layers.Dense(50, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(30, activation="relu"),
#     keras.layers.Dense(20, activation="relu"),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [50]:
rym_cross = sentiment_cross_from_file(4, '../input/big-data-internet-aint-safe/rmdescriptors1000.txt')

In [51]:
# evaluate(rym_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [52]:
# evaluate(rym_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [53]:
# evaluate(rym_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [54]:
# evaluate(rym_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(80, activation="relu"),
#     keras.layers.Dense(70, activation="relu"),
#     keras.layers.Dense(60, activation="relu"),
#     keras.layers.Dense(50, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(30, activation="relu"),
#     keras.layers.Dense(20, activation="relu"),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [55]:
primes_cross = sentiment_cross_from_file(4, '../input/big-data-internet-aint-safe/primes.txt')

In [56]:
# sarcasm_cross = sentiment_cross_from_file(4, '../input/big-data-internet-aint-safe/sarcasm_labelled.txt')

# evaluate(sarcasm_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [57]:
# sarcasm_cross = sentiment_cross_from_file(4, '../input/big-data-internet-aint-safe/sarcasm_labelled.txt')

# evaluate(sarcasm_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [58]:
# sarcasm_cross = sentiment_cross_from_file(4, '../input/big-data-internet-aint-safe/sarcasm_labelled.txt')

# evaluate(sarcasm_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [59]:
# sarcasm_cross = sentiment_cross_from_file(4, '../input/big-data-internet-aint-safe/sarcasm_labelled.txt')

# evaluate(sarcasm_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(80, activation="relu"),
#     keras.layers.Dense(70, activation="relu"),
#     keras.layers.Dense(60, activation="relu"),
#     keras.layers.Dense(50, activation="relu"),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(30, activation="relu"),
#     keras.layers.Dense(20, activation="relu"),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [60]:
drawings_cross = drawing_cross_from_file(4, '../input/drawings/Infinity')


In [61]:
# evaluate_hw(drawings_cross, 40, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [62]:

def evaluate_hw(cross_data: List[Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]], 
                epochs: int, 
                model_maker):
    num_labels = len(set(cross_data[0][0][1]))
    print(f"num_labels: {num_labels}")
    for ((train_examples, train_labels), (test_examples, test_labels)) in cross_data:
        print(f"shape:{train_examples.shape}")
        
        # Convert labels to categorical format
        train_labels_categorical = keras.utils.to_categorical(train_labels, num_labels)
        test_labels_categorical = keras.utils.to_categorical(test_labels, num_labels)
        
        model = model_maker(train_examples.shape[1:], num_labels)
        model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
        history = model.fit(train_examples, train_labels_categorical, epochs=epochs, validation_data=(test_examples, test_labels_categorical))
        
# def evaluate_hw(cross_data: List[Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]], 
#                 epochs: int, 
#                 model_maker):
#     num_labels = len(set(map(tuple, cross_data[0][0][1])))
#     print(f"num_labels: {num_labels}")
#     for ((train_examples, train_labels), (test_examples, test_labels)) in cross_data:
#         print(f"shape:{train_examples.shape}")
#         model = model_maker(train_examples.shape[1:], num_labels)
#         model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
#         history = model.fit(train_examples, train_labels, epochs=epochs, validation_data=(test_examples, test_labels))

# evaluate_hw(drawings_cross, 60, lambda shape, num_labels: keras.models.Sequential([
#     keras.layers.Flatten(input_shape=shape),
#     keras.layers.Dense(40, activation="relu"),
#     keras.layers.Dense(10, activation="relu"),
#     keras.layers.Dense(num_labels, activation="softmax")
# ]))

In [63]:
evaluate_hw(drawings_cross, 400, lambda shape, num_labels: keras.models.Sequential([
    keras.layers.Flatten(input_shape=shape),
    keras.layers.Dense(80, activation="relu"),
    keras.layers.Dense(70, activation="relu"),
    keras.layers.Dense(60, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(num_labels, activation="softmax")
]))

num_labels: 8
shape:(122, 40, 40)
Epoch 1/400
4/4 [==============================] - 1s 100ms/step - loss: 2.0815 - accuracy: 0.1148 - val_loss: 2.0741 - val_accuracy: 0.1220
Epoch 2/400
4/4 [==============================] - 0s 18ms/step - loss: 2.0797 - accuracy: 0.1066 - val_loss: 2.0736 - val_accuracy: 0.1220
Epoch 3/400
4/4 [==============================] - 0s 18ms/step - loss: 2.0781 - accuracy: 0.1230 - val_loss: 2.0734 - val_accuracy: 0.1220
Epoch 4/400
4/4 [==============================] - 0s 16ms/step - loss: 2.0764 - accuracy: 0.1311 - val_loss: 2.0731 - val_accuracy: 0.1220
Epoch 5/400
4/4 [==============================] - 0s 17ms/step - loss: 2.0750 - accuracy: 0.1311 - val_loss: 2.0730 - val_accuracy: 0.0976
Epoch 6/400
4/4 [==============================] - 0s 16ms/step - loss: 2.0738 - accuracy: 0.1311 - val_loss: 2.0728 - val_accuracy: 0.0976
Epoch 7/400
4/4 [==============================] - 0s 16ms/step - loss: 2.0723 - accuracy: 0.1311 - val_loss: 2.0726 - val_ac

In [64]:
evaluate(drawings_cross, 400, lambda shape, num_labels: keras.models.Sequential([
    keras.layers.Flatten(input_shape=shape),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(num_labels, activation="softmax")
]))

shape:(122, 40, 40)
Epoch 1/400
4/4 [==============================] - 1s 113ms/step - loss: 2.0784 - accuracy: 0.1311 - val_loss: 2.0800 - val_accuracy: 0.1463
Epoch 2/400
4/4 [==============================] - 0s 45ms/step - loss: 2.0779 - accuracy: 0.1066 - val_loss: 2.0798 - val_accuracy: 0.1463
Epoch 3/400
4/4 [==============================] - 0s 53ms/step - loss: 2.0771 - accuracy: 0.1230 - val_loss: 2.0794 - val_accuracy: 0.1463
Epoch 4/400
4/4 [==============================] - 0s 44ms/step - loss: 2.0764 - accuracy: 0.1393 - val_loss: 2.0791 - val_accuracy: 0.1707
Epoch 5/400
4/4 [==============================] - 0s 63ms/step - loss: 2.0760 - accuracy: 0.1311 - val_loss: 2.0788 - val_accuracy: 0.1951
Epoch 6/400
4/4 [==============================] - 0s 45ms/step - loss: 2.0752 - accuracy: 0.1639 - val_loss: 2.0785 - val_accuracy: 0.1707
Epoch 7/400
4/4 [==============================] - 0s 29ms/step - loss: 2.0747 - accuracy: 0.1803 - val_loss: 2.0783 - val_accuracy: 0.1707

## Paper

When you are finished with your experiments, write a paper summarizing your findings. Include the following:

* An analysis and discussion of your data. (Be sure to include the data as well.)
  * Again, be sure to use `val_accuracy` as your primary assessment of each neural network, as that reflects the accuracy on the testing data.
* What effect did variations in the number of network layers have?
* How about variations in the number of nodes in each layer?
* Compare the relative difficulty of the two tasks when using neural networks. What aspects of the tasks, in your view, contributed to this relative difficulty?
* How do neural networks compare with k-nearest-neighbor, self-organizing maps, decision trees, and random forests? What are their relative advantages and disadvantages?
* Beyond the actual results, what other issues are noteworthy?

# Level 3: Convolutional Neural Networks

Level 3 of this project is based on [L25 Using Pre-trained Convnet](https://www.kaggle.com/code/jhskaggle/l25-using-pre-trained-convnet), which in turn is based on Chollet, F. (2018). Deep learning with Python. Available at https://www.manning.com/books/deep-learning-with-python. Chollet is the creator of Keras https://keras.io/.

Convolutional neural networks (CNNs) perform extensive preprocessing and feature extraction, and are especially effective for image classification. However, they are extremely expensive to train, as these large models require a lot of data to train.

The VGG16 CNN we will use was trained using the [ImageNet](https://www.image-net.org/) database, which contains over 14 million labeled training images. The model itself has about 138 million parameters.

This model is very popular and is included in Keras. The code block below imports it and creates an instance of it.

In [65]:
from keras.applications.vgg16 import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))
conv_base.trainable = False

58900480/58889256 [==============================] - 3s 0us/step


The above constructor call involves the following parameters:
* `weights='imagenet'` requests the millions of weight values derived from training on the ImageNet database.
* `include_top=False` requests the unsupervised convolutional element of the model but not its classifier. The reason for this is that we want to construct our own classifer atop it.
* `input_shape=(150, 150, 3)` states that the inputs will be 150x150 RGB images.

After calling the constructor, we set `trainable` to `False`. This is to keep the unsupervised elements from being changed as we train our classifier.

Having created the base network, we next import some training data. For this example, we will use a well-known database of dog and cat images to create a dog vs. cat classifier. This database has been pre-split into 2000 training images and 1000 validation images.

In [66]:
from keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = '../input/cats-and-dogs-small/cats_and_dogs_small/train'
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_dir = '../input/cats-and-dogs-small/cats_and_dogs_small/validation'
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Next, we add classification layers to the convolutional layers in order to build the complete network, stored in the `model` variable. We add a hidden layer of 256 nodes and an output layer of one node to serve as our binary classifier.

In [67]:
model = keras.models.Sequential()
model.add(conv_base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])

We are now ready to train our classifier atop the convolutional model. This may take quite a while to run, perhaps 15-20 minutes.

In [68]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=2)

Epoch 1/5
100/100 [==============================] - 208s 2s/step - loss: 0.3899 - acc: 0.8145 - val_loss: 0.1850 - val_acc: 0.9750
Epoch 2/5
100/100 [==============================] - 207s 2s/step - loss: 0.2338 - acc: 0.9065 - val_loss: 0.4084 - val_acc: 0.8000
Epoch 3/5
100/100 [==============================] - 208s 2s/step - loss: 0.1880 - acc: 0.9275 - val_loss: 0.3004 - val_acc: 0.8750
Epoch 4/5
100/100 [==============================] - 206s 2s/step - loss: 0.1548 - acc: 0.9415 - val_loss: 0.3382 - val_acc: 0.8250
Epoch 5/5
100/100 [==============================] - 207s 2s/step - loss: 0.1251 - acc: 0.9565 - val_loss: 0.1413 - val_acc: 0.9500


In [69]:
from keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = '../input/the-woods/images'
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=40,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_dir = '../input/the-woods/images'
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=40,
        class_mode='binary')

Found 366 images belonging to 2 classes.
Found 366 images belonging to 2 classes.


In [70]:
model = keras.models.Sequential()
model.add(conv_base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])

In [71]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=25)

100/100 [==============================] - 175s 2s/step - loss: 0.6955 - acc: 0.6148 - val_loss: 0.5141 - val_acc: 0.7732


In [72]:
from keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = '../input/tomandjerry'
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=40,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_dir = '../input/tomandjerry'
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=40,
        class_mode='binary')



Found 718 images belonging to 2 classes.
Found 718 images belonging to 2 classes.


In [73]:
model = keras.models.Sequential()
model.add(conv_base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])

In [74]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=25)

100/100 [==============================] - 156s 2s/step - loss: 0.4874 - acc: 0.7688 - val_loss: 0.3033 - val_acc: 0.9415


## Assignment

Use the preceding example as inspiration to train two networks:
1. Find a set of labeled images that is of interest to you. Basing your training process on the preceding example, build and assess a network for classifying the images.
2. Take some photos and assemble them into a data set of labeled photographs to upload as a Kaggle data set. Classify them into two labeled groups with 50 photographs per group. Then build a classifier using this same approach, and assess its performance.

## Paper

Add answers to the following questions in your paper:
1. How did VGG16 perform? Compare its accuracy on the dog vs cat problem to the two problems that you studied.
2. Based on your results, for what types of problems do you think VGG16 or similar CNN approaches are most suitable? Discuss the nature of the problem as well as the quantity of data available for training and the time available for training as well.